## TODOs

- [x] take an Excel file and convert it to markdown 
- [  ] then upload to the service 
- [  ] to use File Search
 

# **Sales Analyst Agent**

## **About the Scenario**
This notebook demonstrates a practical use case where an AI Agent is leveraged to gain actionable insights and address key analytical questions related to AdventureWorks sales analysis. It combines file search, computation, and data visualization to streamline decision-making.

The scenario utilizes an AI Agent integrated with two powerful tools: *File Search* and *Code Interpreter*. These tools work together to retrieve sales data from within an Excel Workbook and calculate sales metrics, replicating real-world workflows in retail sales.

### Key Steps:
1. *File Conversion*: Convert Excel Workboot into Markdown tables.
2. *Upload Data*:Import a Markdown file containing the tables into the OpenAI Project.
3. *Perform Sales Analysis*: Leverage *File Search* and *Code Interpreter to compute key sales metrics and insights.
4. *Generate Report*: Leverage *Code Interpreter* to generate sales insights visualizations and leverage Python libraries to render a report.

## **Data**
This scenario uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.

## **Time**
You should expect to spend 10-15 minutes building and running this scenario. 

## **Before you begin**

#### Step 1: Install required libraries
Install dependencies directly within a Jupyter notebook is a good practice because it ensures that all required packages are installed in the correct versions, making the notebook self-contained and reproducible. This approach helps other users or collaborators to set up the environment quickly and avoid potential issues related to missing or incompatible packages.

In [19]:
# Install the packages
%pip install -r ./requirements.txt

print("\nPackages installed successfully.")

Note: you may need to restart the kernel to use updated packages.

Packages installed successfully.


#### Step 2: Setting up the environment
Before we begin, we need to load the necessary environment variables from a `.env` file. These variables include sensitive information such as API keys and endpoint URLs, which are crucial for running the code successfully.

Here’s what you need to do:
- Ensure your `.env` file is properly configured in the `.venv/.env` format. We have provided an template `.env` file, `.env.example` for your reference.
- Verify that all required secrets are included in the file before running the code.


The `.env` file must contain the following secrets:
- PROJECT_CONNECTION_STRING: URL to connect to the Azure OpenAI Project to access project resources.
- AZURE_OPENAI_DEPLOYMENT: The name of the Azure OpenAI model deployment.

Now, let’s load these variables and get started!

<code style="background:yellow;color:black">Note: Make sure to keep your `.env` file secure and avoid sharing it publicly. </code>

*For more information about leveraging Python Virtual Environments can be found [here](https://docs.python.org/3/library/venv.html).*

In [20]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the secrets
__PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
__AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# Verify environment variables
if not all([__PROJECT_CONNECTION_STRING, __AZURE_OPENAI_DEPLOYMENT]):
    raise EnvironmentError("One or more environment variables are missing. Please check the .env file.")
else:
    print("Environment variables loaded successfully.")

Environment variables loaded successfully.


## **Azure OpenAI Agent Setup**

#### Step 1: Initializing the Azure Agent Runtime Client

Next, we’ll initialize the Azure Agent Runtime client. This client allows us to interact with Azure OpenAI Project Agents. We will use a `DefaultAzureCredential` to authenticate, meaning you will have to be logged in with the Azure CLI.

In [21]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Initialize the Azure AI Project client
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

agent_client = project_client.agents

print("Agent client created successfully.")

Agent client created successfully.


### Step 2: Convert Excel workbook to Markdown files
Currently, Microsoft Excel workbooks are not a supported file type for the *File Search* tool, therefore we must convert the Excel Workbook for this scenario into a compatible format, such as Markdown. This process converts each sheet in the Excel file to a Markdown table and saves them as separate Markdown files.

In [22]:
import pandas as pd
import os

# Define paths
output_dir = "uploads"
data_dir_path = "data"
sales_order_files = ["SalesOrder_43659.xlsx", "SalesOrder_43661.xlsx", "SalesOrder_43662.xlsx", "SalesOrder_43665.xlsx"]

output_dir_path = os.path.join(data_dir_path, output_dir)

# Ensure output directory exists
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# Prefix the data directory path to each file
sales_order_files = [os.path.join(data_dir_path, curr_file) for curr_file in sales_order_files]

# Store the uploaded file IDs to be used later when enabling File Search
markdown_file_paths = []

for curr_file in sales_order_files:
    try:
        # Check if the file exists
        if not os.path.exists(curr_file):
            raise FileNotFoundError(f"The file '{curr_file}' does not exist.")

        # Load the Excel File into a DataFrame
        df = pd.read_excel(curr_file)
        print(f"Workbook '{curr_file}' successfully loaded.")

        # Get the base name of the Excel file without the extension
        base_name = os.path.splitext(os.path.basename(curr_file))[0]

        # Convert the DataFrame to a Markdown table
        md_tbl_str = df.to_markdown(index=False, tablefmt="pipe")
        
        # Define the output file name
        output_file = os.path.join(output_dir_path, f"{base_name}.md")
        markdown_file_paths.append(output_file)
        
        # Write the Markdown table to the file
        with open(output_file, "w") as f:
            f.write(md_tbl_str)

        print(f"Markdown file '{output_file}' successfully written.")

    except FileNotFoundError as e:
        print(f"Error: {e}")

    except Exception as e:
        print(f"An error occurred while processing the Excel file '{curr_file}': {e}")


Workbook 'data\SalesOrder_43659.xlsx' successfully loaded.
Markdown file 'data\uploads\SalesOrder_43659.md' successfully written.
Workbook 'data\SalesOrder_43661.xlsx' successfully loaded.
Markdown file 'data\uploads\SalesOrder_43661.md' successfully written.
Workbook 'data\SalesOrder_43662.xlsx' successfully loaded.
Markdown file 'data\uploads\SalesOrder_43662.md' successfully written.
Workbook 'data\SalesOrder_43665.xlsx' successfully loaded.
Markdown file 'data\uploads\SalesOrder_43665.md' successfully written.


### Step 3: Upload supporting file(s) to Azure OpenAI deployment
Now, we'll upload the markdown files in the local directory that we created in the previous step to Azure OpenAI, ensuring any existing markdown files are removed beforehand to ensure the latest version of the files are used and no duplicates exist. This process handles the entire clean up and upload.

In [23]:
uploaded_file_ids = []

try:
    for local_file in os.listdir(output_dir_path):

        # Define the local file path
        local_file_path = os.path.join(output_dir_path, local_file)

        # Check if the file is a Markdown file
        if not local_file.endswith(".md"):
            continue
        else:
            print(f"Processing File '{local_file_path}'...")

            # Check if the file already exists in the cloud, and delete it if it does
            for cloud_file in agent_client.list_files().data:
                if cloud_file.filename == local_file:
                    agent_client.delete_file(cloud_file.id)
                    print(f"Deleted existing cloud file: '{cloud_file.filename}'")

            file = project_client.agents.upload_file_and_poll(file_path=local_file_path, purpose="assistants")
            uploaded_file_ids.append(file.id)
            print(f"Uploaded file, file ID: {file.id}")
            
    print(f"File IDs: {uploaded_file_ids}")

except FileNotFoundError:
    print(f"Error: The file '{curr_file}' was not found.")
    raise

except Exception as e:
    print(f"An error occurred while processing the Excel file: {e}")
    raise

Processing File 'data\uploads\SalesOrder_43659.md'...
Uploaded file, file ID: assistant-gM5qUMC6FFWPUTVN5NtlZJNg
Processing File 'data\uploads\SalesOrder_43661.md'...
Uploaded file, file ID: assistant-IwBAmXH9WVOFVVK9pPgoBBUd
Processing File 'data\uploads\SalesOrder_43662.md'...
Uploaded file, file ID: assistant-JGvJYv0Z61vyFnX14NY3kw2g
Processing File 'data\uploads\SalesOrder_43665.md'...
Uploaded file, file ID: assistant-qPDd0TgBfynHLIFnAcke28UD
File IDs: ['assistant-gM5qUMC6FFWPUTVN5NtlZJNg', 'assistant-IwBAmXH9WVOFVVK9pPgoBBUd', 'assistant-JGvJYv0Z61vyFnX14NY3kw2g', 'assistant-qPDd0TgBfynHLIFnAcke28UD']


### Step 4: Create Vector Store

In [24]:
# Create a vector store called "Financial Statements"
vector_store = agent_client.create_vector_store_and_poll(file_ids=uploaded_file_ids, name="Sales Orders")
print(f"Vector store '{vector_store.name} ({vector_store.id})' created successfully.")

Vector store 'Sales Orders (vs_GH1bM1LM9HHHgLztrkatUPpT)' created successfully.


### Step 5: Create File Search Tool

In [25]:
from azure.ai.projects.models import FileSearchTool

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

## **Azure OpenAI Agent**

vector store gets created when file is uploaded and attached to the message tag as file_search from thread and conversation scope

### Step 1: Create a Sales Analyst Agent

In [26]:
try:
    agent = project_client.agents.create_agent(
        model=__AZURE_OPENAI_DEPLOYMENT,
        name="Sales Analyst Agent",
        instructions=(
            "You are an expert sales analyst. "
            "Use your knowledge base to answer questions about company sales, customers, and products."
        ),
        tools=file_search.definitions,
        tool_resources=file_search.resources,
    )
    print("Agent created successfully.\n", agent.id)
except Exception as e:
    print("Error creating Agent:", e)

Agent created successfully.
 asst_uyBZakeAFO7KBKG09dbH04BU


### Step 2: Starting a New Converstaion Thread

In [27]:
# Create a conversation thread
try:
    thread = agent_client.create_thread()
    print("Thread created successfully:", thread.id)
except Exception as e:
    print("Error creating thread:", e)

Thread created successfully: thread_j6zRse2OYtU3cj27KVcUgXBl


### Step 3: Adding User Message to the Thread

In [28]:
# Define the user question
prompt_content = "How much revenue did we generate in the last quarter by Reseller?"

# Add the question to the thread
try:
    message = agent_client.create_message(
        thread_id=thread.id,
        role="user",
        content=prompt_content,
        #attachments=[{"file_ids": [file for file in uploaded_file_ids]}],
    )
    print("Successfully added User prompt to the thread.\n", message.id)
except Exception as e:
    print("Error adding user question:", e)

Successfully added User prompt to the thread.
 msg_WYimSo7E4ieMvD10IHpfHqIC


### Step 4: Run Agent

In [29]:
# Initiate the Agent's response
try:
    run = agent_client.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent.id,
        #instructions=prompt_content,
    )
    print("Run started:", run.id)
except Exception as e:
    print("Error starting run:", e)

Run started: run_tQDWxRhaSpJ1C38vnr9LNnUe


In [30]:
if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

### Step 5: Agent Response

In [31]:
messages = agent_client.list_messages(thread_id=thread.id, order="asc", after=message.id)

print(f'Run completed!\n\nMESSAGES\n')

# Loop through messages and print content based on role
for msg in messages.data:
    role = msg.role
    content = msg.content[0].text.value
    print(f"{role.capitalize()}: {content}")

Run completed!

MESSAGES

Assistant: The revenue generated in the last quarter by each reseller is as follows:

1. **Original Bicycle Supply Company (Canada)**:
   - Orders: Various products including HL Mountain Frame, Mountain-100 bikes, Sport-100 Helmet, Long-Sleeve Logo Jersey.
   - Total Revenue: $38,750.72【4†source】【2†source】【5†source】【9†source】.

2. **Better Bike Shop (United States, Southeast)**:
   - Orders: Various products including Mountain-100 bikes, Sport-100 Helmet, AWC Logo Cap, Mountain Bike Socks.
   - Total Revenue: $32,294.39【1†source】【6†source】【12†source】.

3. **Latest Sports Equipment (United States, Northwest)**:
   - Orders: Various products including Mountain-100 bikes, Sport-100 Helmet, Long-Sleeve Logo Jersey, AWC Logo Cap, Mountain Bike Socks.
   - Total Revenue: $12,376.86【3†source】【4†source】【11†source】.

4. **Health Spa, Limited (Canada)**:
   - Orders: Various products including LL Road Frame, Road-650 bikes, ML Road Frame, Road-150 bikes.
   - Total Reve

## **Clean Up**

### Step 1: Delete Vector Store

In [32]:
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

Deleted vector store


### Step 3: Delete Files

In [33]:
# For each file id in the list, delete the file
for file_id in uploaded_file_ids:
    project_client.agents.delete_file(file_id)
    print(f"Deleted file: {file_id}")

Deleted file: assistant-gM5qUMC6FFWPUTVN5NtlZJNg
Deleted file: assistant-IwBAmXH9WVOFVVK9pPgoBBUd
Deleted file: assistant-JGvJYv0Z61vyFnX14NY3kw2g
Deleted file: assistant-qPDd0TgBfynHLIFnAcke28UD


### Step 2: Delete Agent

In [34]:
# Get agent list
agents = agent_client.list_agents()

# If the agent exists in the list, delete it
if agent.id in [a.id for a in agents.data]:
    response = agent_client.delete_agent(agent.id)
    print("Deleted Agent Client\n",response)
else:
    print("Agent does not exist to delete.")

Deleted Agent Client
 {'id': 'asst_uyBZakeAFO7KBKG09dbH04BU', 'object': 'assistant.deleted', 'deleted': True}
